
# 🌍 Analyzing Sea Level Rise Using Earth Data in the Cloud

### This notebook is entirely based on Jinbo Wang's [tutorial](https://github.com/betolink/the-coding-club/blob/main/notebooks/Earthdata_webinar_20220727.ipynb)

<img alt="Sea level rise infographic" src="https://www.nasa.gov/wp-content/uploads/2020/08/sealevel_main-causes3-16.jpg" width="800px" />

--- 

**We are going to reproduce the plot from this infographic** Source: [ NASA-led Study Reveals the Causes of Sea Level Rise Since 1900 ](https://grace.jpl.nasa.gov/news/113/nasa-led-study-reveals-the-causes-of-sea-level-rise-since-1900/)


In [ ]:
import earthaccess

print(f"using earthaccess v{earthaccess.__version__}")

auth = earthaccess.login()
# are we authenticated?
if not auth.authenticated:
    # ask for credentials and persist them in a .netrc file
    auth.login(strategy="interactive", persist=True)

### Why earthaccess?

earthaccess is a Python library that simplifies data discovery and access to NASA Earth science data by providing an abstraction layer for NASA’s Common Metadata Repository (CMR) Search API so that searching for data can be done using a simpler notation instead of low level HTTP queries. 

## Authentication in the cloud

If the collection is a cloud-hosted collection we can print the `summary()` and get the S3 credential endpoint. These S3 credentials are temporary and we can use them with third party libraries such as s3fs, boto3 or awscli.

In [ ]:
from pprint import pprint

# We'll get 4 collections that match with our keywords
collections = earthaccess.search_datasets(
    keyword="SEA SURFACE HEIGHT",
    cloud_hosted=True,
    count=4,
)

# Let's print 2 collections
for collection in collections[0:2]:
    # pprint(collection.summary())
    print(
        pprint(collection.summary()),
        collection.abstract(),
        "\n",
        collection["umm"]["DOI"],
        "\n\n",
    )

## A year of data 

Things to keep in mind:

* this particular dataset has data until 2019
* this is a global dataset, each granule represents the whole world
* temporal coverage is 1 granule each 5 days

In [ ]:
granules = earthaccess.search_data(
    short_name="SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL2205",
    count=1,
)

## Working with the URLs directly

If we chose, we can use `earthdata` to grab the file's URLs and then download them with another library (if we have a `.netrc` file configured with NASA's EDL credentials)
Getting the links to our data is quiet simple with the `data_links()` method on each of the results:

In [ ]:
# the collection is cloud hosted, but we can access it out of AWS with the regular HTTPS URL
granules[0].data_links(access="out_of_region")

In [ ]:
granules[0].data_links(access="direct")

## POC: streaming into xarray

We can use `earthaccess` to stream files directly into xarray, this will work well for cases where:

* Data is in NetCDF/HDF5/Zaar format
  * xarray can read bytes directly for remote datasets only with **`h5netcdf`** and **`scipy`** back-ends, if we deal with a format that won't be recognized by these 2 backends xarray will raise an exception.
* Data is not big data (multi TB)
  * not fully tested with Dask distributed
* Data is gridded
  * xarray works better with homogeneous coordinates, working with swath data will be cumbersome.
* Data is chunked using reasonable large sizes (1MB or more)
  * If our files are chunked in small pieces the access time will be orders of magnitude bigger than just downloading the data and accessing it locally.
  
Opening a year of SSH (SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL1812) data (1.1 GB approx) can take up to 5 minutes streaming the data out of region (not in AWS)
The reason for this is not that the data transfer is order of magnitude slower but due the client libraries not fetching data concurrently and the metadata of the files in HDF is usually not consolidated like in Zaar, hence h5netcdf has to issue a lot of requests to get the info it needs.

> Note: we are looping through each year and getting the metadata for the first granule in May

In [ ]:
#  storing the resulting granule metadata
granules = []

# we just grab 1 granule from May for each year of the dataset
for year in range(1999, 2019):
    print(f"Retrieving data for year: {year}")
    results = earthaccess.search_data(
        doi="10.5067/SLREF-CDRV3",
        temporal=(f"{year}-05", f"{year}-05"),
        count=1,
    )
    granules.append(results[0])
print(f"Total granules: {len(granules)}")

### What does `earthaccess.open()` do?

`earthaccess.open()` takes a list of results from `earthaccess.search_data()` or a list of URLs and creates a list of Python File-like objects that can be used in our code as if the remote files were local. When executed in AWS the file system used is [S3FS](https://github.com/fsspec/s3fs) when we open files outside of AWS we get a regular HTTPS file session. 


In [ ]:
%%time

import xarray as xr

fileset = earthaccess.open(granules)

print(f" Using {type(fileset[0])} filesystem")

ds = xr.open_mfdataset(fileset, chunks={})
ds

## Plotting 

In [ ]:
(
    ds.SLA.where((ds.SLA >= 0) & (ds.SLA < 10))
    .std("Time")
    .plot(figsize=(14, 6), x="Longitude", y="Latitude")
)

In [ ]:
import numpy as np
from pyproj import Geod


def ssl_area(lats):
    """Calculate the area associated with a 1/6 by 1/6 degree box at latitude specified in "lats".

    Parameters:
        lats: a list or numpy array of size N the latitudes of interest.

    Returns:
        Array (N) area values (unit: m^2)
    """
    # Define WGS84 as CRS:
    geod = Geod(ellps="WGS84")
    # Set offsets for a box that will be centered at 0, lat
    dx = 1 / 12.0
    lon_offset = np.array((-dx, dx, dx, -dx))
    lat_offset = np.array((-dx, -dx, dx, dx))
    out = []
    for lat in lats:
        # Run a geodesic area calculator for the given box
        c_area, *_ = geod.polygon_area_perimeter(lon_offset, lat + lat_offset)
        out.append(c_area)
    return np.array(out)


# note: they rotated the data in the last release, this operation used to be (1,-1)
ssh_area = ssl_area(ds.Latitude.data).reshape(-1, 1)
print(ssh_area.shape)

In [ ]:
# This dataset was moved from opendap
granule = earthaccess.search_data(concept_id="C2491724765-POCLOUD")[0].data_links()[0]
gmsl = earthaccess.open([granule])[0]

In [ ]:
%%time

from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (16, 4)

img = plt.imread("underwater.jpeg")

fig, axs = plt.subplots()
plt.grid(True)


def global_mean(SLA, **kwargs):
    dout = ((SLA * ssh_area).sum() / (SLA / SLA * ssh_area).sum()) * 1000
    return dout


result = ds.SLA.groupby("Time").apply(global_mean)

plt.xlabel("Time (year)", fontsize=16)
plt.ylabel("Global Mean SLA (meter)", fontsize=12)
# axs.imshow(img, aspect='auto')
plt.grid(True)

historic_ts = xr.open_dataset(gmsl)

result.plot(ax=axs, color="orange", marker="o", label="satellite record")
historic_ts["global_average_sea_level_change"].plot(
    ax=axs, label="Historical in-situ record", color="lightblue"
)

x0, x1 = axs.get_xlim()
y0, y1 = axs.get_ylim()
axs.imshow(img, extent=[x0, x1, y0, y1], aspect="auto")

plt.legend()
plt.show()